In [1]:
#!pip3 install textblob

In [1]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
import configparser as cp
#https://github.com/Refinitiv-API-Samples/Article.EikonAPI.Python.NewsSentimentAnalysis

Basado en el siguiente artículo:

jasonramchandani 
@rsamasiri rsamasiri Raksina Samasiri

#https://github.com/Refinitiv-API-Samples/Article.EikonAPI.Python.NewsSentimentAnalysis

In [2]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

In [3]:
import pandas as pd
pd.merge


<function pandas.core.reshape.merge.merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | None' = None, left_on: 'IndexLabel | None' = None, right_on: 'IndexLabel | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'str | None' = None) -> 'DataFrame'>

In [4]:
from datetime import datetime, timedelta
end = datetime.today()
start = end - timedelta(days=(365.2*4))
print(start, end)

2019-10-03 20:30:56.396580 2023-10-03 15:42:56.396580


In [5]:
df = ek.get_news_headlines('R:MSFT.O AND Language:LEN', date_from=start, date_to=end, count=100)
df.head()

,versionCreated,text,storyId,sourceCode
2023-10-03 15:28:56.268,2023-10-03 15:34:13.969000+00:00,REG - Bango PLC - Total Voting Rights,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE
2023-10-03 10:26:14.160,2023-10-03 15:30:28.389000+00:00,REG - Allianz Global Invs - Top 10 Holdings,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE
2023-10-03 15:29:31.000,2023-10-03 15:29:31+00:00,"BUZZ-Amazon, Microsoft fall as UK regulator pu...",urn:newsml:newswire.refinitiv.com:20231003:nL4...,NS:RTRS
2023-10-03 14:48:08.949,2023-10-03 15:08:15.524000+00:00,REG - Haydale Graphene Ind - Result of General...,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE
2023-10-03 15:00:00.504,2023-10-03 15:00:00.504000+00:00,"ELEVATE SECURITY INTRODUCES ADAPTIVE TRUST, UN...",urn:newsml:newswire.refinitiv.com:20231003:nTU...,NS:RTRS


In [6]:
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan

In [7]:
#for idx, storyId in enumerate(df['storyId'].values):  #for each row in our df dataframe
#    print(idx, storyId)
#    newsText = ek.get_news_story(storyId) #get the news story
#    print(newsText)

In [47]:
#for idx, storyId in enumerate(df['storyId'].values):
#    print(idx, storyId)

In [48]:
newsText = ek.get_news_story('urn:newsml:newswire.refinitiv.com:20230929:nNRAq3au6x:2')
newsText

'<div class="storyContent" lang="en"><style type="text/css">.storyContent * {border-color:inherit !important;outline-color:inherit !important;}</style><div class="tr-npp-dateline"><p>Sep 29, 2023</p></div><div class="tr-npp-lead"><p><span>Microsoft</span>MSFT recently announced that Mayo Clinic, a world leader in healthcare, has become one of the first healthcare organizations to deploy Microsoft 365 Copilot.Microsoft 365 Copilot combines the power of large language models (LLMs) with data in the Microsoft 365 apps, including calendars, emails, chats, documents and meeting transcripts, to turn words into a powerful productivity tool.This new generative artificial intelligence (AI) service combines the power of LLMs with organizational data from Microsoft 365 to enable new levels of productivity in the enterprise. This innovative technology is being tested through the Microsoft 365 Copilot Early Access Program by hundreds of clinical staff, doctors and healthcare workers at Mayo Clinic.

In [52]:
soup = BeautifulSoup(newsText,"lxml")
sentA = TextBlob(soup.get_text())
sentA.sentiment.polarity

0.11182722749886928

In [53]:
for idx, storyId in enumerate(df['storyId'].values):  #for each row in our df dataframe
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
        df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
        if sentA.sentiment.polarity >= 0.05: # attribute bucket to sentiment polartiy
            score = 'positive'
        elif  -.05 < sentA.sentiment.polarity < 0.05:
            score = 'neutral'
        else:
            score = 'negative'
        df['Score'].iloc[idx] = score #write score back to df
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-10-03 15:28:56.268,2023-10-03 15:34:13.969000+00:00,REG - Bango PLC - Total Voting Rights,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.091901,0.397221,positive,NaN,NaN,NaN,NaN
2023-10-03 10:26:14.160,2023-10-03 15:30:28.389000+00:00,REG - Allianz Global Invs - Top 10 Holdings,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.115646,0.342517,positive,NaN,NaN,NaN,NaN
2023-10-03 15:29:31.000,2023-10-03 15:29:31+00:00,"BUZZ-Amazon, Microsoft fall as UK regulator pu...",urn:newsml:newswire.refinitiv.com:20231003:nL4...,NS:RTRS,0.048990,0.407071,neutral,NaN,NaN,NaN,NaN
2023-10-03 14:48:08.949,2023-10-03 15:08:15.524000+00:00,REG - Haydale Graphene Ind - Result of General...,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.015422,0.353602,neutral,NaN,NaN,NaN,NaN
2023-10-03 15:00:00.504,2023-10-03 15:00:00.504000+00:00,"ELEVATE SECURITY INTRODUCES ADAPTIVE TRUST, UN...",urn:newsml:newswire.refinitiv.com:20231003:nTU...,NS:RTRS,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN


In [54]:
start = df['versionCreated'].min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
end = df['versionCreated'].max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
Minute = ek.get_timeseries(["MSFT.O"], start_date=start, interval="minute")
Minute

MSFT.O,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2023-09-29 00:00:00,314.2,313.72,314.12,313.72,48,268
2023-09-29 08:01:00,315.99,314.99,315.99,315.6,17,489
2023-09-29 08:02:00,315.56,315.35,315.56,315.35,9,120
2023-09-29 08:03:00,315.55,315.49,315.51,315.54,19,573
2023-09-29 08:04:00,315.6,315.39,315.5,315.6,18,558
...,...,...,...,...,...,...
2023-10-03 23:42:00,313.7,313.41,313.7,313.57,5,130
2023-10-03 23:43:00,313.58,313.43,313.56,313.43,8,212
2023-10-03 23:44:00,313.7,313.18,313.58,313.7,22,384


In [55]:
df['versionCreated'].min()

Timestamp('2023-09-29 16:22:57.682000+0000', tz='UTC')

In [56]:
df['twoM'] = np.nan
df['fiveM'] = np.nan
df['tenM'] = np.nan
df['thirtyM'] = np.nan
df.head(2)

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-10-03 15:28:56.268,2023-10-03 15:34:13.969000+00:00,REG - Bango PLC - Total Voting Rights,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.091901,0.397221,positive,NaN,NaN,NaN,NaN
2023-10-03 10:26:14.160,2023-10-03 15:30:28.389000+00:00,REG - Allianz Global Invs - Top 10 Holdings,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.115646,0.342517,positive,NaN,NaN,NaN,NaN


In [57]:
df.tail()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-09-29 17:18:59.014,2023-09-29 18:55:16.942000+00:00,The Schall Law Firm Encourages Investors in Ui...,urn:newsml:newsroom.refinitiv.com:20230929:nVM...,NS:CMNW,-0.087368,0.369797,negative,NaN,NaN,NaN,NaN
2023-09-29 18:30:27.000,2023-09-29 18:30:27+00:00,"What to Watch in the Week Ahead and on Monday,...",urn:newsml:newswire.refinitiv.com:20230929:nL4...,NS:RTRS,0.004158,0.376227,neutral,NaN,NaN,NaN,NaN
2023-09-29 18:03:49.000,2023-09-29 18:03:49+00:00,United States Attorney's Office for the Southe...,urn:newsml:newswire.refinitiv.com:20230929:nND...,NS:PUBT,-0.048560,0.449278,neutral,NaN,NaN,NaN,NaN
2023-09-29 16:04:57.725,2023-09-29 17:55:56.554000+00:00,Refinitiv Newscasts - Trading at Noon: Wall St...,urn:newsml:newswire.refinitiv.com:20230929:nRT...,NS:RTRS,0.079630,0.442593,positive,NaN,NaN,NaN,NaN
2023-09-29 16:22:55.323,2023-09-29 16:22:57.682000+00:00,Microsoft (MSFT) Brings 365 Copilot Capabiliti...,urn:newsml:newswire.refinitiv.com:20230929:nNR...,NS:ZACKSC,0.111827,0.543700,positive,NaN,NaN,NaN,NaN


In [13]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    print(idx, newsDate)

0 2023-10-03T15:34:13.969000000
1 2023-10-03T15:30:28.389000000
2 2023-10-03T15:29:31.000000000
3 2023-10-03T15:08:15.524000000
4 2023-10-03T15:00:00.504000000
5 2023-10-03T15:00:00.301000000
6 2023-10-03T14:50:22.697000000
7 2023-10-03T14:47:48.000000000
8 2023-10-03T14:19:21.000000000
9 2023-10-03T14:00:00.000000000
10 2023-10-03T13:58:50.000000000
11 2023-10-03T13:56:16.695000000
12 2023-10-03T13:46:09.000000000
13 2023-10-03T13:43:15.323000000
14 2023-10-03T13:29:26.000000000
15 2023-10-03T13:23:38.000000000
16 2023-10-03T13:00:04.402000000
17 2023-10-03T13:00:04.221000000
18 2023-10-03T12:50:57.000000000
19 2023-10-03T11:56:21.000000000
20 2023-10-03T11:52:57.000000000
21 2023-10-03T11:08:40.000000000
22 2023-10-03T11:07:38.746000000
23 2023-10-03T11:07:02.807000000
24 2023-10-03T11:07:02.566000000
25 2023-10-03T10:36:59.000000000
26 2023-10-03T10:32:02.965000000
27 2023-10-03T10:26:13.000000000
28 2023-10-03T09:29:03.000000000
29 2023-10-03T08:20:17.890000000
30 2023-10-03T08:00:

In [33]:
idx = 0
newsDate = '2023-10-03 22:27:00'
sTime = df['versionCreated'][idx]
sTime = sTime.replace(second=0,microsecond=0).replace(tzinfo=None)
sTime

Timestamp('2023-10-03 15:34:00')

In [30]:
Minute

MSFT.O,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2023-09-29 00:00:00,314.2,313.72,314.12,313.72,48,268
2023-09-29 08:01:00,315.99,314.99,315.99,315.6,17,489
2023-09-29 08:02:00,315.56,315.35,315.56,315.35,9,120
2023-09-29 08:03:00,315.55,315.49,315.51,315.54,19,573
2023-09-29 08:04:00,315.6,315.39,315.5,315.6,18,558
...,...,...,...,...,...,...
2023-10-03 22:26:00,312.85,312.84,312.85,312.84,4,20
2023-10-03 22:27:00,312.85,312.83,312.84,312.85,6,740
2023-10-03 22:28:00,312.89,312.68,312.88,312.89,40,1116


In [34]:
t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
t0

314.03

In [35]:
Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=2))),3]

313.69

In [58]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    sTime = df['versionCreated'][idx]
    sTime = sTime.replace(second=0,microsecond=0)
    try:
        t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
        df['twoM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=2))),3]/(t0)-1)*100)
        df['fiveM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=5))),3]/(t0)-1)*100)
        df['tenM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=10))),3]/(t0)-1)*100) 
        df['thirtyM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + timedelta(minutes=30))),3]/(t0)-1)*100)
    except:
        pass
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-10-03 15:28:56.268,2023-10-03 15:34:13.969000+00:00,REG - Bango PLC - Total Voting Rights,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.091901,0.397221,positive,NaN,NaN,NaN,NaN
2023-10-03 10:26:14.160,2023-10-03 15:30:28.389000+00:00,REG - Allianz Global Invs - Top 10 Holdings,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.115646,0.342517,positive,NaN,NaN,NaN,NaN
2023-10-03 15:29:31.000,2023-10-03 15:29:31+00:00,"BUZZ-Amazon, Microsoft fall as UK regulator pu...",urn:newsml:newswire.refinitiv.com:20231003:nL4...,NS:RTRS,0.048990,0.407071,neutral,NaN,NaN,NaN,NaN
2023-10-03 14:48:08.949,2023-10-03 15:08:15.524000+00:00,REG - Haydale Graphene Ind - Result of General...,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.015422,0.353602,neutral,NaN,NaN,NaN,NaN
2023-10-03 15:00:00.504,2023-10-03 15:00:00.504000+00:00,"ELEVATE SECURITY INTRODUCES ADAPTIVE TRUST, UN...",urn:newsml:newswire.refinitiv.com:20231003:nTU...,NS:RTRS,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN


In [59]:
df

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-10-03 15:28:56.268,2023-10-03 15:34:13.969000+00:00,REG - Bango PLC - Total Voting Rights,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.091901,0.397221,positive,NaN,NaN,NaN,NaN
2023-10-03 10:26:14.160,2023-10-03 15:30:28.389000+00:00,REG - Allianz Global Invs - Top 10 Holdings,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.115646,0.342517,positive,NaN,NaN,NaN,NaN
2023-10-03 15:29:31.000,2023-10-03 15:29:31+00:00,"BUZZ-Amazon, Microsoft fall as UK regulator pu...",urn:newsml:newswire.refinitiv.com:20231003:nL4...,NS:RTRS,0.048990,0.407071,neutral,NaN,NaN,NaN,NaN
2023-10-03 14:48:08.949,2023-10-03 15:08:15.524000+00:00,REG - Haydale Graphene Ind - Result of General...,urn:newsml:newsroom.refinitiv.com:20231003:nRS...,NS:LSE,0.015422,0.353602,neutral,NaN,NaN,NaN,NaN
2023-10-03 15:00:00.504,2023-10-03 15:00:00.504000+00:00,"ELEVATE SECURITY INTRODUCES ADAPTIVE TRUST, UN...",urn:newsml:newswire.refinitiv.com:20231003:nTU...,NS:RTRS,0.000000,0.000000,neutral,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-29 17:18:59.014,2023-09-29 18:55:16.942000+00:00,The Schall Law Firm Encourages Investors in Ui...,urn:newsml:newsroom.refinitiv.com:20230929:nVM...,NS:CMNW,-0.087368,0.369797,negative,NaN,NaN,NaN,NaN
2023-09-29 18:30:27.000,2023-09-29 18:30:27+00:00,"What to Watch in the Week Ahead and on Monday,...",urn:newsml:newswire.refinitiv.com:20230929:nL4...,NS:RTRS,0.004158,0.376227,neutral,NaN,NaN,NaN,NaN
2023-09-29 18:03:49.000,2023-09-29 18:03:49+00:00,United States Attorney's Office for the Southe...,urn:newsml:newswire.refinitiv.com:20230929:nND...,NS:PUBT,-0.048560,0.449278,neutral,NaN,NaN,NaN,NaN
2023-09-29 16:04:57.725,2023-09-29 17:55:56.554000+00:00,Refinitiv Newscasts - Trading at Noon: Wall St...,urn:newsml:newswire.refinitiv.com:20230929:nRT...,NS:RTRS,0.079630,0.442593,positive,NaN,NaN,NaN,NaN


In [42]:
grouped = df.groupby(['Score']).mean()
grouped

TypeError: Cannot perform reduction 'mean' with string dtype